<a href="https://colab.research.google.com/github/zganjei/translate-text-to-sql/blob/main/translate_text_to_sql.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Text to SQL translator

This is demo of a Python-based tool that leverages the power of OpenAI's GPT to transform a natural language prompt into an SQL query. This tool can enable a company's consultants to be able to interact with client data more efficiently. This allows them to generate insightful reports and recommendations for their clients without the actual knowledge of SQL language.

## Install necessary packages and setup the environment

In [34]:
%%capture
!pip install openai
!pip install pandas
!pip install sqlalchemy


import openai
import pandas as pd
from sqlalchemy import create_engine, text

from google.colab import userdata

api_key = userdata.get('openai.api_key')
if api_key:
  print(f"API key: {api_key[:20]}....")
else:
  print("API key not found!")
openai.api_key = api_key
from google.colab import drive
drive.mount("/content/drive")


Load the customer data to be analyzed and store it in a data frame

In [12]:
csv_file_path = "/content/drive/My Drive/Colab Notebooks/customer_orders.csv"
data_frame = pd.read_csv(csv_file_path)
data_frame.head()

,ORDERNUMBER,QUANTITYORDERED,PRICEEACH,ORDERLINENUMBER,SALES,ORDERDATE,STATUS,QTR_ID,MONTH_ID,YEAR_ID,...,ADDRESSLINE1,ADDRESSLINE2,CITY,STATE,POSTALCODE,COUNTRY,TERRITORY,CONTACTLASTNAME,CONTACTFIRSTNAME,DEALSIZE
0,10107,30,95.70,2,2871.00,2/24/2003 0:00,Shipped,1,2,2003,...,897 Long Airport Avenue,NaN,NYC,NY,10022,USA,NaN,Yu,Kwai,Small
1,10121,34,81.35,5,2765.90,5/7/2003 0:00,Shipped,2,5,2003,...,59 rue de l'Abbaye,NaN,Reims,NaN,51100,France,EMEA,Henriot,Paul,Small
2,10134,41,94.74,2,3884.34,7/1/2003 0:00,Shipped,3,7,2003,...,27 rue du Colonel Pierre Avia,NaN,Paris,NaN,75508,France,EMEA,Da Cunha,Daniel,Medium
3,10145,45,83.26,6,3746.70,8/25/2003 0:00,Shipped,3,8,2003,...,78934 Hillside Dr.,NaN,Pasadena,CA,90003,USA,NaN,Young,Julie,Medium
4,10159,49,100.00,14,5205.27,10/10/2003 0:00,Shipped,4,10,2003,...,7734 Strong St.,NaN,San Francisco,CA,NaN,USA,NaN,Brown,Julie,Medium


Create an engine from Sqlalchemy and then use that engine to create a database table from the data frame. We'll later use this database table to run SQL query on it

In [13]:
engine = create_engine('sqlite:///:memory:', echo=True)
data_frame.to_sql("SalesTable",engine)

2025-02-23 13:15:02,079 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-02-23 13:15:02,084 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("SalesTable")


INFO:sqlalchemy.engine.Engine:PRAGMA main.table_info("SalesTable")


2025-02-23 13:15:02,086 INFO sqlalchemy.engine.Engine [raw sql] ()


...

2025-02-23 13:15:02,157 INFO sqlalchemy.engine.Engine [raw sql] ()


INFO:sqlalchemy.engine.Engine:[raw sql] ()


2025-02-23 13:15:02,159 INFO sqlalchemy.engine.Engine COMMIT


INFO:sqlalchemy.engine.Engine:COMMIT


2823

Define a function to execute a give SQL query on the table

In [14]:
def execute_sql_query(connection, query):
  result_obj = connection.execute(text(query))
  return result_obj.fetchall()

Create part of the prompt from creating the table schema

In [39]:
def table_schema_prompt(df):
  # get the columns of the dataframe itself and store it in a string
  columns = ", ".join(df.columns)
  return f"### SQLite Table Schema:\n# SalesTable({columns})\n"

def get_user_input():
  return input("Enter your query:")

def full_prompt(df, user_prompt):
  schema_prompt = table_schema_prompt(df)
  # instruct GPT to begin our prompt with SELECT
  return f"{schema_prompt}### Query to answer: {user_prompt}\nSELECT"

user_prompt = get_user_input()
complete_prompt = full_prompt(data_frame, user_prompt)

print(complete_prompt)

Enter your query:give total sales of motorcyles in territory EMEA
### SQLite Table Schema:
# SalesTable(ORDERNUMBER, QUANTITYORDERED, PRICEEACH, ORDERLINENUMBER, SALES, ORDERDATE, STATUS, QTR_ID, MONTH_ID, YEAR_ID, PRODUCTLINE, MSRP, PRODUCTCODE, CUSTOMERNAME, PHONE, ADDRESSLINE1, ADDRESSLINE2, CITY, STATE, POSTALCODE, COUNTRY, TERRITORY, CONTACTLASTNAME, CONTACTFIRSTNAME, DEALSIZE)
### Query to answer: give total sales of motorcyles in territory EMEA
SELECT


Pass the prompt to GPT to get SQL code

In [40]:
response = openai.chat.completions.create(
    model = "gpt-3.5-turbo",
    messages=[{"role": "system", "content": "You give only sql query text, nothing more."},
              {"role": "user", "content": complete_prompt}],
    temperature=0,
    max_tokens=150
)

def process_response(res):
  # get the response and remove the white spaces
  raw_query = res.choices[0].message.content.strip()
  if not raw_query.upper().startswith("SELECT"):
    raw_query = f"SELECT {raw_query}"
  return raw_query

Create a connection to the engine and run the query on the database table

In [41]:
final_query = process_response(response)
with engine.connect() as conn:
  final_result = execute_sql_query(conn, final_query)
  print(f"\nfinal result: {final_result}\n")


2025-02-23 13:33:52,774 INFO sqlalchemy.engine.Engine BEGIN (implicit)


INFO:sqlalchemy.engine.Engine:BEGIN (implicit)


2025-02-23 13:33:52,775 INFO sqlalchemy.engine.Engine SELECT SUM(SALES) 
FROM SalesTable 
WHERE PRODUCTLINE = 'Motorcycles' AND TERRITORY = 'EMEA';


INFO:sqlalchemy.engine.Engine:SELECT SUM(SALES) 
FROM SalesTable 
WHERE PRODUCTLINE = 'Motorcycles' AND TERRITORY = 'EMEA';


2025-02-23 13:33:52,776 INFO sqlalchemy.engine.Engine [generated in 0.00208s] ()


INFO:sqlalchemy.engine.Engine:[generated in 0.00208s] ()



final result: [(503096.7,)]

2025-02-23 13:33:52,779 INFO sqlalchemy.engine.Engine ROLLBACK


INFO:sqlalchemy.engine.Engine:ROLLBACK
